In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#############################################################################################################################################################################
#################################################### ESCOLARIDADE, EMPREGABILIDADE E ALCOOLISMO #############################################################################
#############################################################################################################################################################################

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

spark = SparkSession.builder.appName("GetLinesEstatisticasPopul").getOrCreate()

# Caminhos para os arquivos das três tabelas Delta Lake
hdfs_file_path_continentes= "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/continentes_table"
delta_estatPopul_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/estatPopul_table"

df_continentes = spark.read.format("delta").load(hdfs_file_path_continentes)
df_EstatPopul = spark.read.format("delta").load(delta_estatPopul_path)

# Filtrar as linhas onde a coluna IndicatorName é igual a 'School enrollment, secondary (% net)' ou igual a 'Share of youth not in education, employment or training, total (% of youth population)'
df_EstatPopul = df_EstatPopul.filter((df_EstatPopul['IndicatorName'] == 'Gross graduation ratio, tertiary, female (%)') |
                                     (df_EstatPopul['IndicatorName'] == 'Gross graduation ratio, tertiary, male (%)') |
                                     (df_EstatPopul['IndicatorName'] == 'Gross graduation ratio, tertiary, total (%)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, tertiary (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, tertiary, female (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, tertiary, male (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, primary (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, primary, female (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, primary, male (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, secondary (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, secondary, female (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'School enrollment, secondary, male (% gross)') |
                                     (df_EstatPopul['IndicatorName'] == 'Prevalence of overweight (% of adults)')|
                                     (df_EstatPopul['IndicatorName'] == 'Prevalence of overweight, female (% of female adults)')|
                                     (df_EstatPopul['IndicatorName'] == 'Prevalence of overweight, male (% of male adults)'))

#######################################################################################################################################################################
#######################################################################################################################################################################

# Caminhos para os arquivos das duas tabelas Delta Lake
delta_desemprego_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/desemprego_table"
delta_consumoAlcool_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/consumoAlcool_table"

# Ler as duas tabelas Delta Lake
df_desemprego = spark.read.format("delta").load(delta_desemprego_path)

#eliminar colunas
columns_to_drop = ['CountryCode']
df_desemprego = df_desemprego.drop(*columns_to_drop)

df_consumoAlcool = spark.read.format("delta").load(delta_consumoAlcool_path)

#rename colunas
df_consumoAlcool = df_consumoAlcool.withColumnRenamed("DeathPerAnnum", "TotalDeathPerAnnum") 

# Realizar o join entre os DataFrames usando as colunas em comum 'CountryName' e 'Year'
df_EmpregAlcool = df_desemprego.join(df_consumoAlcool, ['CountryName', 'Year'], 'inner')

#######################################################################################################################################################################
#######################################################################################################################################################################

# Realizar o join entre os DataFrames usando as colunas em comum 'CountryName' e 'Year'
df_EscolEmpregAlcool = df_EstatPopul.join(df_EmpregAlcool, ['CountryName', 'Year'], 'inner')

df_EscolEmpregAlcool = df_EscolEmpregAlcool.join(df_continentes, ['CountryName'], 'inner')

# Defina a ordem desejada das colunas
nova_ordem_colunas = ['CountryCode', 'CountryName', 'RegionName', 'SubRegionName', 'IndicatorCode', 'IndicatorName', 'Year', 'IndicatorValue', 'Population', 'UnemploymentRate', 'TotalDeathPerAnnum', 'AlcoholConsumptionPerPersonInLiter', 'DeathDueToAlcohol']

# Reordene as colunas do DataFrame df_EscolEmpregAlcool
df_EscolEmpregAlcool = df_EscolEmpregAlcool.select(nova_ordem_colunas)

# Calculando a nova coluna 'PercentageDeathDueToAlcohol'
df_EscolEmpregAlcool = df_EscolEmpregAlcool.withColumn(
    'PercentageDeathDueToAlcohol',
    (col('DeathDueToAlcohol') / col('TotalDeathPerAnnum')) * 100
)


df_EscolEmpregAlcool.toPandas()

CountryCode CountryName RegionName                    SubRegionName  \
0            MEX      Mexico   Americas  Latin America and the Caribbean   
1            MEX      Mexico   Americas  Latin America and the Caribbean   
2            MEX      Mexico   Americas  Latin America and the Caribbean   
3            MEX      Mexico   Americas  Latin America and the Caribbean   
4            MEX      Mexico   Americas  Latin America and the Caribbean   
...          ...         ...        ...                              ...   
7693         ZMB      Zambia     Africa               Sub-Saharan Africa   
7694         ZMB      Zambia     Africa               Sub-Saharan Africa   
7695         ZMB      Zambia     Africa               Sub-Saharan Africa   
7696         ZMB      Zambia     Africa               Sub-Saharan Africa   
7697         ZMB      Zambia     Africa               Sub-Saharan Africa   

          IndicatorCode                                      IndicatorName  \
0        SE.TER.ENRR.MA        School enrollment, tertiary, male (% gross)   
1        SE.TER.ENRR.FE      School enrollment, tertiary, female (% gross)   
2           SE.TER.ENRR              School enrollment, tertiary (% gross)   
3        SE.SEC.ENRR.MA       School enrollment, secondary, male (% gross)   
4        SE.SEC.ENRR.FE     School enrollment, secondary, female (% gross)   
...                 ...                                                ...   
7693     SE.PRM.ENRR.FE       School enrollment, primary, female (% gross)   
7694        SE.PRM.ENRR               School enrollment, primary (% gross)   
7695  SH.STA.OWAD.MA.ZS  Prevalence of overweight, male (% of male adults)   
7696  SH.STA.OWAD.FE.ZS  Prevalence of overweight, female (% of female ...   
7697     SH.STA.OWAD.ZS             Prevalence of overweight (% of adults)   

      Year  IndicatorValue  Population UnemploymentRate  TotalDeathPerAnnum  \
0     2000       21.524099      150331             2.65               31708   
1     2000       19.505529      150331             2.65               31708   
2     2000       20.491011      150331             2.65               31708   
3     2000       71.142021      150331             2.65               31708   
4     2000       72.393356      150331             2.65               31708   
...    ...             ...         ...              ...                 ...   
7693  2010      107.313583    13605986            13.19                5413   
7694  2010      106.978767    13605986            13.19                5413   
7695  2010       16.600000    13605986            13.19                5413   
7696  2010       32.100000    13605986            13.19                5413   
7697  2010       24.500000    13605986            13.19                5413   

      AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  \
0                                   6.43               4696   
1                                   6.43               4696   
2                                   6.43               4696   
3                                   6.43               4696   
4                                   6.43               4696   
...                                  ...                ...   
7693                                5.43                 99   
7694                                5.43                 99   
7695                                5.43                 99   
7696                                5.43                 99   
7697                                5.43                 99   

      PercentageDeathDueToAlcohol  
0                       14.810143  
1                       14.810143  
2                       14.810143  
3                       14.810143  
4                       14.810143  
...                           ...  
7693                     1.828930  
7694                     1.828930  
7695                     1.828930  
7696                     1.828930  
7697                     1.828930  

[7698 rows x 14 

In [30]:
df_EscolEmpregAlcool.createOrReplaceTempView("EscolEmpregoAlcool_table")

# Executar a consulta SQL para selecionar os valores onde CountryName é 'Spain' e Year é '2015'
query = """
    SELECT *
    FROM EscolEmpregoAlcool_table
    WHERE CountryName = 'Mexico' and Year = '2015'
"""

# Executar a consulta SQL e exibir o resultado
df_result = spark.sql(query)
df_result.toPandas()

CountryCode CountryName      IndicatorCode  \
0          MEX      Mexico     SE.TER.ENRR.MA   
1          MEX      Mexico     SE.TER.ENRR.FE   
2          MEX      Mexico        SE.TER.ENRR   
3          MEX      Mexico     SE.SEC.ENRR.MA   
4          MEX      Mexico     SE.SEC.ENRR.FE   
5          MEX      Mexico        SE.SEC.ENRR   
6          MEX      Mexico     SE.PRM.ENRR.MA   
7          MEX      Mexico     SE.PRM.ENRR.FE   
8          MEX      Mexico        SE.PRM.ENRR   
9          MEX      Mexico     SE.TER.CMPL.ZS   
10         MEX      Mexico  SE.TER.CMPL.MA.ZS   
11         MEX      Mexico  SE.TER.CMPL.FE.ZS   

                                     IndicatorName  Year  IndicatorValue  \
0      School enrollment, tertiary, male (% gross)  2015       32.071442   
1    School enrollment, tertiary, female (% gross)  2015       31.563620   
2            School enrollment, tertiary (% gross)  2015       31.819040   
3     School enrollment, secondary, male (% gross)  2015       97.649918   
4   School enrollment, secondary, female (% gross)  2015      104.093781   
5           School enrollment, secondary (% gross)  2015      100.832741   
6       School enrollment, primary, male (% gross)  2015      106.735077   
7     School enrollment, primary, female (% gross)  2015      106.471863   
8             School enrollment, primary (% gross)  2015      106.606110   
9      Gross graduation ratio, tertiary, total (%)  2015       22.076710   
10      Gross graduation ratio, tertiary, male (%)  2015       20.337339   
11    Gross graduation ratio, tertiary, female (%)  2015       23.830099   

    Population UnemploymentRate  TotalDeathPerAnnum  \
0    121858251             4.31               41021   
1    121858251             4.31               41021   
2    121858251             4.31               41021   
3    121858251             4.31               41021   
4    121858251             4.31               41021   
5    121858251             4.31               41021   
6    121858251             4.31               41021   
7    121858251             4.31               41021   
8    121858251             4.31               41021   
9    121858251             4.31               41021   
10   121858251             4.31               41021   
11   121858251             4.31               41021   

    AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  \
0                                 4.92               4048   
1                                 4.92               4048   
2                                 4.92               4048   
3                                 4.92               4048   
4                                 4.92               4048   
5                                 4.92               4048   
6                                 4.92               4048   
7                                 4.92               4048   
8                                 4.92               4048   
9                                 4.92               4048   
10                                4.92               4048   
11                                4.92               4048   

    PercentageDeathDueToAlcohol  
0                      9.868116  
1                      9.868116  
2                      9.868116  
3                      9.868116  
4                      9.868116  
5                      9.868116  
6                      9.868116  
7                      9.868116  
8                      9.868116  
9                      9.868116  
10                     9.868116  
11                     9.868116

In [6]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
EscolEmpregoAlcool_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/gold.db/EscolEmpregoAlcool_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, EscolEmpregoAlcool_table)

# Seu DataFrame com os novos dados
df = df_EscolEmpregAlcool

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(EscolEmpregoAlcool_table)

In [20]:
spark.sql(
    """
    SELECT *
    FROM gold.EscolEmpregoAlcool_table
    """
).toPandas()

CountryCode CountryName RegionName                    SubRegionName  \
0            ARG   Argentina   Americas  Latin America and the Caribbean   
1            ARG   Argentina   Americas  Latin America and the Caribbean   
2            ARG   Argentina   Americas  Latin America and the Caribbean   
3            ARG   Argentina   Americas  Latin America and the Caribbean   
4            ARG   Argentina   Americas  Latin America and the Caribbean   
...          ...         ...        ...                              ...   
7693         COM     Comoros     Africa               Sub-Saharan Africa   
7694         COM     Comoros     Africa               Sub-Saharan Africa   
7695         COM     Comoros     Africa               Sub-Saharan Africa   
7696         COM     Comoros     Africa               Sub-Saharan Africa   
7697         COM     Comoros     Africa               Sub-Saharan Africa   

       IndicatorCode                                   IndicatorName  \
0     SE.TER.ENRR.MA     School enrollment, tertiary, male (% gross)   
1     SE.TER.ENRR.FE   School enrollment, tertiary, female (% gross)   
2        SE.TER.ENRR           School enrollment, tertiary (% gross)   
3     SE.SEC.ENRR.MA    School enrollment, secondary, male (% gross)   
4     SE.SEC.ENRR.FE  School enrollment, secondary, female (% gross)   
...              ...                                             ...   
7693  SE.SEC.ENRR.FE  School enrollment, secondary, female (% gross)   
7694     SE.SEC.ENRR          School enrollment, secondary (% gross)   
7695  SE.PRM.ENRR.MA      School enrollment, primary, male (% gross)   
7696  SE.PRM.ENRR.FE    School enrollment, primary, female (% gross)   
7697     SE.PRM.ENRR            School enrollment, primary (% gross)   

      IndicatorValue  Population UnemploymentRate  TotalDeathPerAnnum  \
0          58.882019    40895751             7.71               17750   
1          87.828278    40895751             7.71               17750   
2          73.231400    40895751             7.71               17750   
3          94.995041    40895751             7.71               17750   
4         104.154480    40895751             7.71               17750   
...              ...         ...              ...                 ...   
7693       61.414509      832322             8.08                  67   
7694       59.468861      832322             8.08                  67   
7695       99.587578      832322             8.08                  67   
7696       99.422791      832322             8.08                  67   
7697       99.506630      832322             8.08                  67   

      AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  \
0                                   9.09                775   
1                                   9.09                775   
2                                   9.09                775   
3                                   9.09                775   
4                                   9.09                775   
...                                  ...                ...   
7693                                0.69                  6   
7694                                0.69                  6   
7695                                0.69                  6   
7696                                0.69                  6   
7697                                0.69                  6   

      PercentageDeathDueToAlcohol  Year  
0                        4.366197  2010  
1                        4.366197  2010  
2                        4.366197  2010  
3                        4.366197  2010  
4                        4.366197  2010  
...                           ...   ...  
7693                     8.955224  2018  
7694                     8.955224  2018  
7695                     8.955224  2018  
7696                     8.955224  2018  
7697                     8.955224  2018  

[7698 rows x 14 columns]

In [9]:
spark.sql(
    """
    SELECT *
    FROM gold.EscolEmpregoAlcool_table
    where IndicatorName = 'Prevalence of overweight, female (% of female adults)'
    """
).toPandas()

CountryCode               CountryName RegionName  \
0           ARG                 Argentina   Americas   
1           GAB                     Gabon     Africa   
2           AGO                    Angola     Africa   
3           ZWE                  Zimbabwe     Africa   
4           ARM                   Armenia       Asia   
..          ...                       ...        ...   
595         MNE                Montenegro     Europe   
596         BWA                  Botswana     Africa   
597         NPL                     Nepal       Asia   
598         CAF  Central African Republic     Africa   
599         WSM                     Samoa    Oceania   

                       SubRegionName      IndicatorCode  \
0    Latin America and the Caribbean  SH.STA.OWAD.FE.ZS   
1                 Sub-Saharan Africa  SH.STA.OWAD.FE.ZS   
2                 Sub-Saharan Africa  SH.STA.OWAD.FE.ZS   
3                 Sub-Saharan Africa  SH.STA.OWAD.FE.ZS   
4                       Western Asia  SH.STA.OWAD.FE.ZS   
..                               ...                ...   
595                  Southern Europe  SH.STA.OWAD.FE.ZS   
596               Sub-Saharan Africa  SH.STA.OWAD.FE.ZS   
597                    Southern Asia  SH.STA.OWAD.FE.ZS   
598               Sub-Saharan Africa  SH.STA.OWAD.FE.ZS   
599                        Polynesia  SH.STA.OWAD.FE.ZS   

                                         IndicatorName  IndicatorValue  \
0    Prevalence of overweight, female (% of female ...            56.9   
1    Prevalence of overweight, female (% of female ...            43.5   
2    Prevalence of overweight, female (% of female ...            30.3   
3    Prevalence of overweight, female (% of female ...            49.5   
4    Prevalence of overweight, female (% of female ...            51.5   
..                                                 ...             ...   
595  Prevalence of overweight, female (% of female ...            45.7   
596  Prevalence of overweight, female (% of female ...            46.7   
597  Prevalence of overweight, female (% of female ...            13.7   
598  Prevalence of overweight, female (% of female ...            24.0   
599  Prevalence of overweight, female (% of female ...            74.5   

     Population UnemploymentRate  TotalDeathPerAnnum  \
0      40895751             7.71               17750   
1       1624146            20.39                 811   
2      23356247             9.43               10471   
3      12697728             5.21                4939   
4       2877314            19.01                 907   
..          ...              ...                 ...   
595      613558            28.59                 458   
596     1643333            15.88                1124   
597    23941099             1.86                3214   
598     3640421             5.61                1862   
599      174454             4.66                  29   

     AlcoholConsumptionPerPersonInLiter  DeathDueToAlcohol  \
0                                  9.09                775   
1                                  9.89                 17   
2                                  8.16                158   
3                                  3.93                 46   
4                                  5.69                 62   
..                                  ...                ...   
595                                 NaN                  8   
596                                9.05                 23   
597                                1.12                148   
598                                3.68                 44   
599                                3.79                  2   

     PercentageDeathDueToAlcohol  Year  
0                       4.366197  2010  
1                       2.096178  2010  
2                       1.508929  2010  
3                       0.931363  2010  
4                       6.835722  2010  
..                           ...   ...  
595                     1.74

In [ ]:
#############################################################################################################################################################################
###################################################################### COVID E HIV ##########################################################################################
#############################################################################################################################################################################

In [21]:
# Importar bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit


# Carregar os dataframes
hdfs_file_path_continentes= "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/continentes_table" 
hdfs_file_path_est_pop = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/estatPopul_table"
hdfs_file_path_covid = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/covid_table"

df_continentes = spark.read.format("delta").load(hdfs_file_path_continentes)
df_est_pop = spark.read.format("delta").load(hdfs_file_path_est_pop)
df_covid = spark.read.format("delta").load(hdfs_file_path_covid)


###################################################################### Editar COVID #########################################################################################

#eliminar colunas
columns_to_drop = ['SerialNumber', 'Population']
df_covid = df_covid.drop(*columns_to_drop)

###################################################################### BEFORE COVID #########################################################################################
# Filtrar as linhas onde a coluna IndicatorName é igual a 
df_est_pop = df_est_pop.filter((df_est_pop['IndicatorName'] == 'Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)') |
                                    (df_est_pop['IndicatorName'] == 'Incidence of HIV, ages 15-49, female (per 1,000 uninfected female population ages 15-49)') |
                                    (df_est_pop['IndicatorName'] == 'Incidence of HIV, ages 15-49, male (per 1,000 uninfected male population ages 15-49)'))

# Filtrar os dados para considerar apenas os anos de 2013 a 2019
filtered_years = df_est_pop[df_est_pop['Year'].between(2013, 2019)]

# Group them by airdate and name, calculating the average and sum of number of episodes
df_est_pop_avg = filtered_years \
    .groupBy('CountryCode', 'CountryName', 'IndicatorName') \
    .agg(
        avg(filtered_years.IndicatorValue).alias("AvgIndicatorValue_BeforeCovid")
    )

# Adicionar uma nova coluna chamada "PeriodOfTime" com valores iguais a 2013-2019 usando lit()
df_CovidHIV1 = df_est_pop_avg.withColumn("PeriodOfTime_BeforeCovid", lit('2013-2019'))

###################################################################### AFTER COVID ##########################################################################################

# Carregar os dataframes
hdfs_file_path_est_pop2 = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/estatPopul_table"

df_est_pop2 = spark.read.format("delta").load(hdfs_file_path_est_pop2)

# Filtrar as linhas onde a coluna IndicatorName é igual a 
df_est_pop2 = df_est_pop2.filter((df_est_pop2['IndicatorName'] == 'Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)') |
                                    (df_est_pop2['IndicatorName'] == 'Incidence of HIV, ages 15-49, female (per 1,000 uninfected female population ages 15-49)') |
                                    (df_est_pop2['IndicatorName'] == 'Incidence of HIV, ages 15-49, male (per 1,000 uninfected male population ages 15-49)'))

# Filtrar os dados para considerar apenas os anos de 2020 a 2022
filtered_years2 = df_est_pop2[df_est_pop2['Year'].between(2020, 2022)]

# Group them by airdate and name, calculating the average and sum of number of episodes
df_est_pop_avg2 = filtered_years2 \
    .groupBy('CountryCode', 'CountryName', 'IndicatorName') \
    .agg(
        avg(filtered_years2.IndicatorValue).alias("AvgIndicatorValue_AfterCovid")
    )

# Adicionar uma nova coluna chamada "PeriodOfTime" com valores iguais a 2020-2022 usando lit()
df_CovidHIV2 = df_est_pop_avg2.withColumn("PeriodOfTime_AfterCovid", lit('2020-2022'))

###################################################################### BEFORE AND AFTER COVID ##########################################################################################

# Realizar o join entre os DataFrames usando as colunas em comum 'CountryName' e 'Year'
df_CovidHIV = df_CovidHIV1.join(df_CovidHIV2, ['CountryCode','CountryName', 'IndicatorName'], 'inner')

# Defina a ordem desejada das colunas
ordem_colunas = ['CountryCode', 'CountryName', 'IndicatorName', 'PeriodOfTime_BeforeCovid', 'AvgIndicatorValue_BeforeCovid', 'PeriodOfTime_AfterCovid', 'AvgIndicatorValue_AfterCovid']

# Reordene as colunas do DataFrame df_CovidHIV
df_CovidHIV = df_CovidHIV.select(ordem_colunas)

###################################################################### df_CovidHIV com o df_covid ##########################################################################################

# Juntar os dataframes para ter valores do covid
df_CovidHIV = df_CovidHIV.join(df_covid, ['CountryName'], 'inner')

df_CovidHIV = df_CovidHIV.join(df_continentes, ['CountryName'], 'inner')

# Defina a ordem desejada das colunas
ordem_colunas = ['CountryCode', 'CountryName', 'RegionName', 'SubRegionName', 'IndicatorName', 'PeriodOfTime_BeforeCovid', 'AvgIndicatorValue_BeforeCovid', 'PeriodOfTime_AfterCovid', 'AvgIndicatorValue_AfterCovid', 'TotalCovidCases', 'TotalCovidDeaths', 'TotalCovidRecovered', 'TotalCovidTest']

# Reordene as colunas do DataFrame df_CovidHIV
df_CovidHIV = df_CovidHIV.select(ordem_colunas)

df_CovidHIV.toPandas()

CountryCode CountryName RegionName                    SubRegionName  \
0           HND    Honduras   Americas  Latin America and the Caribbean   
1           KEN       Kenya     Africa               Sub-Saharan Africa   
2           GUY      Guyana   Americas  Latin America and the Caribbean   
3           GAB       Gabon     Africa               Sub-Saharan Africa   
4           GTM   Guatemala   Americas  Latin America and the Caribbean   
..          ...         ...        ...                              ...   
349         ZMB      Zambia     Africa               Sub-Saharan Africa   
350         ZWE    Zimbabwe     Africa               Sub-Saharan Africa   
351         ZMB      Zambia     Africa               Sub-Saharan Africa   
352         ZWE    Zimbabwe     Africa               Sub-Saharan Africa   
353         ZWE    Zimbabwe     Africa               Sub-Saharan Africa   

                                         IndicatorName  \
0    Incidence of HIV, ages 15-49, female (per 1,00...   
1    Incidence of HIV, ages 15-49, male (per 1,000 ...   
2    Incidence of HIV, ages 15-49, male (per 1,000 ...   
3    Incidence of HIV, ages 15-49, female (per 1,00...   
4    Incidence of HIV, ages 15-49, female (per 1,00...   
..                                                 ...   
349  Incidence of HIV, ages 15-49, female (per 1,00...   
350  Incidence of HIV, ages 15-49, female (per 1,00...   
351  Incidence of HIV, ages 15-49 (per 1,000 uninfe...   
352  Incidence of HIV, ages 15-49 (per 1,000 uninfe...   
353  Incidence of HIV, ages 15-49, male (per 1,000 ...   

    PeriodOfTime_BeforeCovid  AvgIndicatorValue_BeforeCovid  \
0                  2013-2019                       0.038571   
1                  2013-2019                       1.104286   
2                  2013-2019                       0.974286   
3                  2013-2019                       2.250000   
4                  2013-2019                       0.024286   
..                       ...                            ...   
349                2013-2019                      10.005714   
350                2013-2019                       6.961429   
351                2013-2019                       7.761429   
352                2013-2019                       5.597143   
353                2013-2019                       4.297143   

    PeriodOfTime_AfterCovid  AvgIndicatorValue_AfterCovid  TotalCovidCases  \
0                 2020-2022                         0.030           470757   
1                 2020-2022                         0.705           342810   
2                 2020-2022                         1.060            72950   
3                 2020-2022                         1.760            48981   
4                 2020-2022                         0.060          1227853   
..                      ...                           ...              ...   
349               2020-2022                         5.850           340763   
350               2020-2022                         3.340           261606   
351               2020-2022                         4.345           340763   
352               2020-2022                         2.550           261606   
353               2020-2022                         1.800           261606   

     TotalCovidDeaths  TotalCovidRecovered  TotalCovidTest  
0               11104                  NaN       1619676.0  
1                5688             337040.0       3967062.0  
2                1294              71357.0        717867.0  
3                 306              48674.0       1621909.0  
4               20106            1204545.0       6757892.0  
..                ...                  ...             ...  
349              4047             335104.0       3865993.0  
350              5652             254951.0       2525756.0  
351              4047             335104.0       3865993.0  
352              5652             254951.0       2525756.0  
353              5652             254951.

In [22]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
CovidHIV_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/gold.db/CovidHIV_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, CovidHIV_table)

# Seu DataFrame com os novos dados
df = df_CovidHIV

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(CovidHIV_table)

In [23]:
spark.sql(
    """
    SELECT *
    FROM gold.CovidHIV_table
    """
).toPandas()

CountryName CountryCode RegionName                    SubRegionName  \
0      Honduras         HND   Americas  Latin America and the Caribbean   
1         Kenya         KEN     Africa               Sub-Saharan Africa   
2        Guyana         GUY   Americas  Latin America and the Caribbean   
3         Gabon         GAB     Africa               Sub-Saharan Africa   
4     Guatemala         GTM   Americas  Latin America and the Caribbean   
..          ...         ...        ...                              ...   
349      Zambia         ZMB     Africa               Sub-Saharan Africa   
350    Zimbabwe         ZWE     Africa               Sub-Saharan Africa   
351      Zambia         ZMB     Africa               Sub-Saharan Africa   
352    Zimbabwe         ZWE     Africa               Sub-Saharan Africa   
353    Zimbabwe         ZWE     Africa               Sub-Saharan Africa   

                                         IndicatorName  \
0    Incidence of HIV, ages 15-49, female (per 1,00...   
1    Incidence of HIV, ages 15-49, male (per 1,000 ...   
2    Incidence of HIV, ages 15-49, male (per 1,000 ...   
3    Incidence of HIV, ages 15-49, female (per 1,00...   
4    Incidence of HIV, ages 15-49, female (per 1,00...   
..                                                 ...   
349  Incidence of HIV, ages 15-49, female (per 1,00...   
350  Incidence of HIV, ages 15-49, female (per 1,00...   
351  Incidence of HIV, ages 15-49 (per 1,000 uninfe...   
352  Incidence of HIV, ages 15-49 (per 1,000 uninfe...   
353  Incidence of HIV, ages 15-49, male (per 1,000 ...   

    PeriodOfTime_BeforeCovid  AvgIndicatorValue_BeforeCovid  \
0                  2013-2019                       0.038571   
1                  2013-2019                       1.104286   
2                  2013-2019                       0.974286   
3                  2013-2019                       2.250000   
4                  2013-2019                       0.024286   
..                       ...                            ...   
349                2013-2019                      10.005714   
350                2013-2019                       6.961429   
351                2013-2019                       7.761429   
352                2013-2019                       5.597143   
353                2013-2019                       4.297143   

    PeriodOfTime_AfterCovid  AvgIndicatorValue_AfterCovid  TotalCovidCases  \
0                 2020-2022                         0.030           470757   
1                 2020-2022                         0.705           342810   
2                 2020-2022                         1.060            72950   
3                 2020-2022                         1.760            48981   
4                 2020-2022                         0.060          1227853   
..                      ...                           ...              ...   
349               2020-2022                         5.850           340763   
350               2020-2022                         3.340           261606   
351               2020-2022                         4.345           340763   
352               2020-2022                         2.550           261606   
353               2020-2022                         1.800           261606   

     TotalCovidDeaths  TotalCovidRecovered  TotalCovidTest  
0               11104                  NaN       1619676.0  
1                5688             337040.0       3967062.0  
2                1294              71357.0        717867.0  
3                 306              48674.0       1621909.0  
4               20106            1204545.0       6757892.0  
..                ...                  ...             ...  
349              4047             335104.0       3865993.0  
350              5652             254951.0       2525756.0  
351              4047             335104.0       3865993.0  
352              5652             254951.0       2525756.0  
353              5652             254951.

In [2]:
spark.sql(
    """
    SELECT *
    FROM gold.CovidHIV_table
    where CountryName = 'Eswatini'
    """
).toPandas()

CountryName CountryCode RegionName       SubRegionName  \
0    Eswatini         SWZ     Africa  Sub-Saharan Africa   
1    Eswatini         SWZ     Africa  Sub-Saharan Africa   
2    Eswatini         SWZ     Africa  Sub-Saharan Africa   

                                       IndicatorName PeriodOfTime_BeforeCovid  \
0  Incidence of HIV, ages 15-49, male (per 1,000 ...                2013-2019   
1  Incidence of HIV, ages 15-49, female (per 1,00...                2013-2019   
2  Incidence of HIV, ages 15-49 (per 1,000 uninfe...                2013-2019   

   AvgIndicatorValue_BeforeCovid PeriodOfTime_AfterCovid  \
0                      17.031429               2020-2022   
1                      35.511429               2020-2022   
2                      25.778571               2020-2022   

   AvgIndicatorValue_AfterCovid  TotalCovidCases  TotalCovidDeaths  \
0                         9.595              NaN               NaN   
1                        22.680              NaN               NaN   
2                        15.525              NaN               NaN   

   TotalCovidRecovered  TotalCovidTest  
0                  NaN             NaN  
1                  NaN             NaN  
2                  NaN             NaN

In [ ]:
#############################################################################################################################################################################
######################################################## ECONOMIA, MORTALIDADE E SAÚDE ######################################################################################
#############################################################################################################################################################################

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, when
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("GetLinesDataSets").getOrCreate()

# Caminhos para os arquivos da tabela Delta Lake estatsPopul
hdfs_file_path_continentes= "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/continentes_table"
delta_estatPopul_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/estatPopul_table"

df_continentes = spark.read.format("delta").load(hdfs_file_path_continentes)
df_EstatPopul = spark.read.format("delta").load(delta_estatPopul_path)

# Filtrar as linhas onde a coluna IndicatorName é igual a 
df_EstatPopul = df_EstatPopul.filter((df_EstatPopul['IndicatorName'] == 'Death rate, crude (per 1,000 people)') |
                                    (df_EstatPopul['IndicatorName'] == 'Mortality rate, adult, male (per 1,000 male adults)') |
                                    (df_EstatPopul['IndicatorName'] == 'Mortality rate, adult, female (per 1,000 female adults)')|
                                    (df_EstatPopul['IndicatorName'] == 'Suicide mortality rate (per 100,000 population)')|
                                    (df_EstatPopul['IndicatorName'] == 'Suicide mortality rate, female (per 100,000 female population)')|
                                    (df_EstatPopul['IndicatorName'] == 'Suicide mortality rate, male (per 100,000 female population)'))

#############################################################################################################################################################################
#############################################################################################################################################################################

# Caminhos para os arquivos da tabela Delta Lake salarioMinimo
delta_table_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/salMin_table"

# Para ler uma tabela Delta Lake
dfGold_SalarioMinimo = spark.read.format("delta").load(delta_table_path)

# Remove as linhas onde PayPeriod é "Hourly"
dfGold_SalarioMinimo = dfGold_SalarioMinimo.filter(dfGold_SalarioMinimo["PayPeriod"] != "Hourly")

#criar um dataframe com os valores mensais apenas
dfMonthly = dfGold_SalarioMinimo.withColumn(
    "Value",
    when(col("PayPeriod") == 'Annual', col("Value") / 12).otherwise(col("Value"))
)

#renomear o atributo Annual para Monthly
dfMonthly = dfMonthly.withColumn("PayPeriod", F.when(F.col("PayPeriod") == 'Annual', 'Monthly').otherwise(F.col("PayPeriod")))

#unir o data frame com os valores anuais e mensais
dfGold_SalarioMinimo = dfGold_SalarioMinimo.union(dfMonthly)

#eliminar colunas
columns_to_drop = ['CountryCode']
dfGold_SalarioMinimo = dfGold_SalarioMinimo.drop(*columns_to_drop)


#############################################################################################################################################################################
#############################################################################################################################################################################

# Caminhos para os arquivos das duas tabelas Delta Lake desemprego
delta_desemprego_path = "hdfs://hdfs-nn:9000/projeto_EDSTD/silver.db/desemprego_table"

# Ler a tabela Delta Lake
df_desemprego = spark.read.format("delta").load(delta_desemprego_path)

#eliminar colunas
columns_to_drop = ['CountryCode']
df_desemprego = df_desemprego.drop(*columns_to_drop)

#############################################################################################################################################################################
#############################################################################################################################################################################

#uniao de todos os dataframes pela coluna CountryName e Year

dfEconoMortSaude = df_EstatPopul.join(dfGold_SalarioMinimo, ['CountryName', 'Year'], 'inner') \
    .join(df_desemprego, ['CountryName', 'Year'], 'inner') 

#rename colunas
dfEconoMortSaude = dfEconoMortSaude.withColumnRenamed("Series", "SalarySeries") \
                                   .withColumnRenamed("PayPeriod", "SalaryPayPeriod") \
                                   .withColumnRenamed("Value", "SalaryValue") \
                                   .withColumnRenamed("DeathPerAnnum", "TotalDeathPerAnnum") \
                                   .withColumnRenamed("UnitCode", "SalaryUnitCode") 

dfEconoMortSaude = dfEconoMortSaude.join(df_continentes, ['CountryName'], 'inner')

# Defina a ordem desejada das colunas
ordem_colunas = ['CountryCode', 'CountryName', 'RegionName', 'SubRegionName', 'IndicatorCode', 'IndicatorName', 'Year', 'IndicatorValue', 'UnemploymentRate', 'SalarySeries', 'SalaryPayPeriod', 'SalaryUnitCode', 'SalaryValue']

# Reordene as colunas do DataFrame df_EscolEmpregAlcool
dfEconoMortSaude = dfEconoMortSaude.select(ordem_colunas)

dfEconoMortSaude.toPandas()

CountryCode CountryName RegionName              SubRegionName  \
0             CAN      Canada   Americas           Northern America   
1             CAN      Canada   Americas           Northern America   
2             CAN      Canada   Americas           Northern America   
3             CAN      Canada   Americas           Northern America   
4             CAN      Canada   Americas           Northern America   
...           ...         ...        ...                        ...   
12039         AUS   Australia    Oceania  Australia and New Zealand   
12040         AUS   Australia    Oceania  Australia and New Zealand   
12041         AUS   Australia    Oceania  Australia and New Zealand   
12042         AUS   Australia    Oceania  Australia and New Zealand   
12043         AUS   Australia    Oceania  Australia and New Zealand   

        IndicatorCode                                      IndicatorName  \
0      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
1      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
2      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
3      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
4      SP.DYN.AMRT.FE  Mortality rate, adult, female (per 1,000 femal...   
...               ...                                                ...   
12039  SP.DYN.AMRT.FE  Mortality rate, adult, female (per 1,000 femal...   
12040  SP.DYN.AMRT.MA  Mortality rate, adult, male (per 1,000 male ad...   
12041  SP.DYN.AMRT.MA  Mortality rate, adult, male (per 1,000 male ad...   
12042  SP.DYN.AMRT.MA  Mortality rate, adult, male (per 1,000 male ad...   
12043  SP.DYN.AMRT.MA  Mortality rate, adult, male (per 1,000 male ad...   

       Year  IndicatorValue UnemploymentRate  \
0      2015           7.400             6.91   
1      2015           7.400             6.91   
2      2015           7.400             6.91   
3      2015           7.400             6.91   
4      2015          50.736             6.91   
...     ...             ...              ...   
12039  2020          41.652             6.46   
12040  2020          73.018             6.46   
12041  2020          73.018             6.46   
12042  2020          73.018             6.46   
12043  2020          73.018             6.46   

                                            SalarySeries SalaryPayPeriod  \
0      In 2022 constant prices at 2022 USD exchange r...         Monthly   
1               In 2022 constant prices at 2022 USD PPPs         Monthly   
2      In 2022 constant prices at 2022 USD exchange r...          Annual   
3               In 2022 constant prices at 2022 USD PPPs          Annual   
4      In 2022 constant prices at 2022 USD exchange r...         Monthly   
...                                                  ...             ...   
12039  In 2022 constant prices at 2022 USD exchange r...          Annual   
12040           In 2022 constant prices at 2022 USD PPPs         Monthly   
12041  In 2022 constant prices at 2022 USD exchange r...         Monthly   
12042           In 2022 constant prices at 2022 USD PPPs          Annual   
12043  In 2022 constant prices at 2022 USD exchange r...          Annual   

      SalaryUnitCode   SalaryValue  
0                USD   1699.585154  
1                USD   1686.672900  
2                USD  20395.021851  
3                USD  20240.074803  
4                USD   1699.585154  
...              ...           ...  
12039            USD  29555.269677  
12040            USD   2317.031764  
12041            USD   2462.939140  
12042            USD  27804.381162  
12043            USD  29555.269677  

[12044 rows x 13 columns]

In [13]:
df_EstatPopul.createOrReplaceTempView("EconoMortSaude_table")

# Executar a consulta SQL para selecionar os valores onde CountryName é 'Spain' e Year é '2015'
query = """
    SELECT *
    FROM EconoMortSaude_table
    WHERE IndicatorName = 'Suicide mortality rate (per 100,000 population)'
"""

# Executar a consulta SQL e exibir o resultado
df_result = spark.sql(query)
df_result.toPandas()

CountryName CountryCode                                    IndicatorName  \
0        Burundi         BDI  Suicide mortality rate (per 100,000 population)   
1     Cabo Verde         CPV  Suicide mortality rate (per 100,000 population)   
2       Cambodia         KHM  Suicide mortality rate (per 100,000 population)   
3       Cameroon         CMR  Suicide mortality rate (per 100,000 population)   
4         Canada         CAN  Suicide mortality rate (per 100,000 population)   
...          ...         ...                                              ...   
4395    Zimbabwe         ZWE  Suicide mortality rate (per 100,000 population)   
4396      Zambia         ZMB  Suicide mortality rate (per 100,000 population)   
4397    Zimbabwe         ZWE  Suicide mortality rate (per 100,000 population)   
4398      Zambia         ZMB  Suicide mortality rate (per 100,000 population)   
4399    Zimbabwe         ZWE  Suicide mortality rate (per 100,000 population)   

       IndicatorCode  IndicatorValue  Year  
0     SH.STA.SUIC.P5             6.4  2015  
1     SH.STA.SUIC.P5            14.4  2015  
2     SH.STA.SUIC.P5             5.1  2015  
3     SH.STA.SUIC.P5             9.5  2015  
4     SH.STA.SUIC.P5            13.0  2015  
...              ...             ...   ...  
4395  SH.STA.SUIC.P5            12.8  2001  
4396  SH.STA.SUIC.P5             9.7  2008  
4397  SH.STA.SUIC.P5            20.1  2008  
4398  SH.STA.SUIC.P5            11.6  2003  
4399  SH.STA.SUIC.P5            13.2  2003  

[4400 rows x 6 columns]

In [16]:
from delta import DeltaTable
from pyspark.sql import SparkSession

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("Write to Delta Lake").getOrCreate()

# Caminho da tabela Delta no HDFS
EconoMortSaude_table = "hdfs://hdfs-nn:9000/projeto_EDSTD/gold.db/EconoMortSaude_table"

# Carregue a tabela Delta Lake existente
delta_table = DeltaTable.forPath(spark, EconoMortSaude_table)

# Seu DataFrame com os novos dados
df = dfEconoMortSaude

# Escreva o DataFrame na tabela Delta Lake existente
df.write.format("delta").mode("append").save(EconoMortSaude_table)

In [17]:
spark.sql(
    """
    SELECT *
    FROM gold.EconoMortSaude_table
    """
).toPandas()

CountryCode CountryName RegionName              SubRegionName  \
0             CAN      Canada   Americas           Northern America   
1             CAN      Canada   Americas           Northern America   
2             CAN      Canada   Americas           Northern America   
3             CAN      Canada   Americas           Northern America   
4             CAN      Canada   Americas           Northern America   
...           ...         ...        ...                        ...   
12039         AUS   Australia    Oceania  Australia and New Zealand   
12040         ESP       Spain     Europe            Southern Europe   
12041         ESP       Spain     Europe            Southern Europe   
12042         ESP       Spain     Europe            Southern Europe   
12043         ESP       Spain     Europe            Southern Europe   

        IndicatorCode                                      IndicatorName  \
0      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
1      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
2      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
3      SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
4      SP.DYN.AMRT.FE  Mortality rate, adult, female (per 1,000 femal...   
...               ...                                                ...   
12039  SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
12040  SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
12041  SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
12042  SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   
12043  SP.DYN.CDRT.IN               Death rate, crude (per 1,000 people)   

       IndicatorValue UnemploymentRate  \
0               7.400             6.91   
1               7.400             6.91   
2               7.400             6.91   
3               7.400             6.91   
4              50.736             6.91   
...               ...              ...   
12039           6.700             5.11   
12040           9.500            14.73   
12041           9.500            14.73   
12042           9.500            14.73   
12043           9.500            14.73   

                                            SalarySeries SalaryPayPeriod  \
0      In 2022 constant prices at 2022 USD exchange r...         Monthly   
1               In 2022 constant prices at 2022 USD PPPs         Monthly   
2      In 2022 constant prices at 2022 USD exchange r...          Annual   
3               In 2022 constant prices at 2022 USD PPPs          Annual   
4      In 2022 constant prices at 2022 USD exchange r...         Monthly   
...                                                  ...             ...   
12039  In 2022 constant prices at 2022 USD exchange r...          Annual   
12040           In 2022 constant prices at 2022 USD PPPs         Monthly   
12041  In 2022 constant prices at 2022 USD exchange r...         Monthly   
12042           In 2022 constant prices at 2022 USD PPPs          Annual   
12043  In 2022 constant prices at 2022 USD exchange r...          Annual   

      SalaryUnitCode   SalaryValue  Year  
0                USD   1699.585154  2015  
1                USD   1686.672900  2015  
2                USD  20395.021851  2015  
3                USD  20240.074803  2015  
4                USD   1699.585154  2015  
...              ...           ...   ...  
12039            USD  29343.129343  2021  
12040            USD   1744.423856  2021  
12041            USD   1265.057420  2021  
12042            USD  20933.086267  2021  
12043            USD  15180.689045  2021  

[12044 rows x 13 columns]

In [19]:
spark.sql(
    """
    SELECT *
    FROM gold.EconoMortSaude_table
    where IndicatorName = 'Suicide mortality rate (per 100,000 population)'
    """
).toPandas()

CountryCode CountryName RegionName                    SubRegionName  \
0            CAN      Canada   Americas                 Northern America   
1            CAN      Canada   Americas                 Northern America   
2            CAN      Canada   Americas                 Northern America   
3            CAN      Canada   Americas                 Northern America   
4            CHL       Chile   Americas  Latin America and the Caribbean   
...          ...         ...        ...                              ...   
2295         ESP       Spain     Europe                  Southern Europe   
2296         ESP       Spain     Europe                  Southern Europe   
2297         ESP       Spain     Europe                  Southern Europe   
2298         ESP       Spain     Europe                  Southern Europe   
2299         ESP       Spain     Europe                  Southern Europe   

       IndicatorCode                                    IndicatorName  \
0     SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
1     SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
2     SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
3     SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
4     SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
...              ...                                              ...   
2295  SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
2296  SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
2297  SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
2298  SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   
2299  SH.STA.SUIC.P5  Suicide mortality rate (per 100,000 population)   

      IndicatorValue UnemploymentRate  \
0               13.0             6.91   
1               13.0             6.91   
2               13.0             6.91   
3               13.0             6.91   
4               10.0             6.51   
...              ...              ...   
2295             7.8            15.25   
2296             8.6            24.44   
2297             8.6            24.44   
2298             8.6            24.44   
2299             8.6            24.44   

                                           SalarySeries SalaryPayPeriod  \
0     In 2022 constant prices at 2022 USD exchange r...         Monthly   
1              In 2022 constant prices at 2022 USD PPPs         Monthly   
2     In 2022 constant prices at 2022 USD exchange r...          Annual   
3              In 2022 constant prices at 2022 USD PPPs          Annual   
4              In 2022 constant prices at 2022 USD PPPs         Monthly   
...                                                 ...             ...   
2295  In 2022 constant prices at 2022 USD exchange r...          Annual   
2296           In 2022 constant prices at 2022 USD PPPs         Monthly   
2297  In 2022 constant prices at 2022 USD exchange r...         Monthly   
2298           In 2022 constant prices at 2022 USD PPPs          Annual   
2299  In 2022 constant prices at 2022 USD exchange r...          Annual   

     SalaryUnitCode   SalaryValue  Year  
0               USD   1699.585154  2015  
1               USD   1686.672900  2015  
2               USD  20395.021851  2015  
3               USD  20240.074803  2015  
4               USD    614.802030  2015  
...             ...           ...   ...  
2295            USD  12168.578825  2018  
2296            USD   1262.150634  2014  
2297            USD    915.312538  2014  
2298            USD  15145.807613  2014  
2299            USD  10983.750450  2014  

[2300 rows x 13 columns]